In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### **Sentence → Classification → Vector Embedding → Meaning → Contextual Meaning**

#### **Contextual meaning:**
Used to capture how the same word can have different meanings in different contexts. This helps generate distinct vector embeddings for the same word depending on its usage and surrounding words.

In [5]:
import warnings

warnings.filterwarnings("ignore")

In [6]:
import tensorflow_hub as hub
import tensorflow_text as text

#### Each encoder model has its own specific preprocessor, which must be used whenever performing any operation with that encoder.

In [7]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [8]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

2025-11-02 12:40:41.305038: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
text_set = ['nice movie indeed', 'I love python programming', 'I got confused with typescript and javascript']

text_preprocess = bert_preprocess_model(text_set)
text_preprocess.keys()

dict_keys(['input_mask', 'input_type_ids', 'input_word_ids'])

### **text_preprocess**, this object has different attributes, lets analyse one by one

In [13]:
text_preprocess['input_mask']

# 128 is the maximum length of the vector that can be generated
# 3 means for 3 sentences it generated 3 vector embedding

<tf.Tensor: shape=(3, 128), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0

In [14]:
text_preprocess['input_type_ids']

<tf.Tensor: shape=(3, 128), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0

In [15]:
text_preprocess['input_word_ids']

<tf.Tensor: shape=(3, 128), dtype=int32, numpy=
array([[  101,  3835,  3185,  5262,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [19]:
bert_model = hub.KerasLayer(encoder_url)

bert_results = bert_model(text_preprocess)
bert_results.keys()

dict_keys(['default', 'encoder_outputs', 'pooled_output', 'sequence_output'])

In [20]:
bert_results['encoder_outputs']

# output of each ecoder layer in the bert model
# 3 -> sentences
# 128 -> embedding
# 768 -> sequence output



[<tf.Tensor: shape=(3, 128, 768), dtype=float32, numpy=
 array([[[ 0.12901433,  0.00644742, -0.03614973, ...,  0.04999627,
           0.06149201, -0.02657528],
         [ 1.1753385 ,  1.2140789 ,  1.1569982 , ...,  0.11634358,
          -0.3585536 , -0.4049018 ],
         [ 0.03859036,  0.5386998 , -0.21089786, ...,  0.21858224,
           0.7260168 , -1.1158603 ],
         ...,
         [-0.0758702 , -0.25421908,  0.7075512 , ...,  0.50542   ,
          -0.1887868 ,  0.15028337],
         [-0.16066599, -0.28089687,  0.5759707 , ...,  0.52758545,
          -0.11141391,  0.0288756 ],
         [-0.04428162, -0.20279588,  0.59093565, ...,  0.8133835 ,
          -0.39075804, -0.02601745]],
 
        [[ 0.1890359 ,  0.02752538, -0.06513739, ..., -0.00620209,
           0.15053897,  0.03165445],
         [ 0.5916149 ,  0.75891405, -0.07240669, ...,  0.6190399 ,
           0.82928926,  0.16161972],
         [ 1.4460827 ,  0.4460263 ,  0.40990257, ...,  0.48255897,
           0.6269115 ,  0.13

In [21]:
bert_results['sequence_output']

<tf.Tensor: shape=(3, 128, 768), dtype=float32, numpy=
array([[[ 0.07292046,  0.08567815,  0.14476833, ..., -0.09677089,
          0.08722138,  0.07711098],
        [ 0.17839392, -0.19006105,  0.50349474, ..., -0.05869799,
          0.32717094, -0.15578488],
        [ 0.18701468, -0.43388787, -0.48875138, ..., -0.15502772,
          0.00145124, -0.24470966],
        ...,
        [ 0.12083106,  0.12884258,  0.46453524, ...,  0.07375515,
          0.17441994,  0.16522008],
        [ 0.07967864, -0.01190711,  0.50225407, ...,  0.13777758,
          0.21002217,  0.00624588],
        [-0.07212694, -0.2830348 ,  0.59033334, ...,  0.47551954,
          0.16668473, -0.08920345]],

       [[-0.07900596,  0.36335123, -0.2110157 , ..., -0.17183766,
          0.1629974 ,  0.6724269 ],
        [ 0.27883503,  0.43716323, -0.35764694, ..., -0.0446365 ,
          0.3831516 ,  0.5887988 ],
        [ 1.203767  ,  1.0727018 ,  0.4840876 , ...,  0.24920997,
          0.40730938,  0.404818  ],
        ...,

# BERT on TensorFlow Hub — Step-by-step explanation



## 1. Goal

Understand what each TensorFlow Hub BERT artifact does and how to interpret the tensors returned by the preprocessing layer and the encoder. After reading this you should know:

* What `bert_preprocess_model` returns (`input_word_ids`, `input_mask`, `input_type_ids`).
* What the BERT encoder outputs (`sequence_output`, `pooled_output`, `encoder_outputs`, `default`).
* How to use `sequence_output` vs `pooled_output` for different downstream tasks (classification, token labeling, retrieval, etc.).


## 2. Links (useful references)

* List of BERT models on TF Hub: [https://tfhub.dev/google/collections/bert/1](https://tfhub.dev/google/collections/bert/1)
* BERT (uncased, base) info: [https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4)


## 3. Minimal code (your snippet)

```python
import tensorflow_hub as hub
import tensorflow_text as text

preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess_model = hub.KerasLayer(preprocess_url)
text_test = ['nice movie indeed','I love python programming']
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed.keys()
# dict_keys(['input_mask', 'input_type_ids', 'input_word_ids'])
```

> The preprocessing layer tokenizes text, adds special tokens, and returns integer tensors ready for the BERT encoder.



## 4. What the preprocessing outputs mean

When you print `text_preprocessed.keys()` you get three tensors:

* `input_word_ids` — token ids (integers) for each token in each input sequence. These are the tokenized IDs the encoder expects. The sequences are padded to a fixed max length (here 128).

* `input_mask` — a mask with `1` for real tokens and `0` for padding tokens. This tells the model which positions are meaningful.

* `input_type_ids` (also called `segment ids`) — used to distinguish sentence A vs sentence B when BERT is used for paired-sentence tasks (e.g., question-answer, next-sentence prediction). For single-sentence inputs this is usually all zeros.

Example shapes from your run:

* Each tensor has shape `(batch_size, max_seq_len)` → here `(2, 128)`.
* `input_word_ids` contains special token ids at the start and end: `101` is `[CLS]` and `102` is `[SEP]` in the WordPiece vocabulary used by BERT.

> Quick note: `101` (CLS) is placed at the beginning of every input. `102` (SEP) ends a single sentence or separates sentence A and B.


## 5. Load the encoder and inspect outputs

```python
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_preprocessed)
bert_results.keys()
# dict_keys(['default', 'encoder_outputs', 'pooled_output', 'sequence_output'])
```

These keys are common across TF Hub BERT models. Here's what each entry contains:

* `sequence_output` — a tensor of shape `(batch_size, seq_len, hidden_size)` (here `(2, 128, 768)`). It contains the contextualized embedding vector for **every token** in the input sequence. Use this for token-level tasks like NER, POS tagging, or any task where you need a vector per token.

* `pooled_output` — a tensor of shape `(batch_size, hidden_size)` (here `(2, 768)`). This is typically the embedding corresponding to the `[CLS]` token after an extra pooling step (in the original BERT implementation it's passed through a small tanh activation layer). Use this as a fixed-length sentence embedding for classification tasks (sentiment, intent, etc.).

* `encoder_outputs` — a list (or tuple) of length `L` where each element is the sequence output from that encoder layer. For BERT base, `len(encoder_outputs) == 12`. Each element has shape `(batch_size, seq_len, hidden_size)` and corresponds to the output after each Transformer block. The last item in this list equals `sequence_output`.

* `default` — some TF Hub models include a `default` key that is often the most convenient top-level object (it commonly maps to the pooled output or another default tensor). Check model docs for exact behaviour; prefer explicit keys (`sequence_output`, `pooled_output`) for clarity.

You observed:

```python
bert_results['encoder_outputs'][-1] == bert_results['sequence_output']
# returns True (element-wise equality)
```

That shows the final encoder layer output is the same tensor as `sequence_output`.


## 6. How to use these outputs in common tasks

* **Sentence classification (sentiment, intent, etc.)**

  * Use `pooled_output` as a feature vector. Feed it to one or more Dense layers plus a softmax/sigmoid head.
  * Example: `Dense(256, activation='relu')(pooled_output) -> Dense(num_classes, activation='softmax')`

* **Token classification (NER, chunking)**

  * Use `sequence_output` to get a vector per token. Pass that through a `TimeDistributed(Dense(...))` or a token-level classifier.
  * Remember to ignore padded tokens using `input_mask` when computing loss and metrics.

* **Semantic search / embeddings**

  * You can use `pooled_output` or a custom pooling over `sequence_output` (mean pooling over non-padded tokens) to get sentence embeddings. If you need strong semantic embeddings, consider fine-tuning or using models trained specifically for sentence embeddings.

* **Layer-wise features**

  * Use `encoder_outputs[i]` to experiment: lower layers capture syntactic features, middle layers mix syntax/semantics, and higher layers capture higher-level semantics useful for classification. Some practitioners concatenate or average several top layers for improved performance.


## 7. Small code snippets you’ll likely use

**Get CLS embedding (pooled) for classification**

```python
cls_embeddings = bert_results['pooled_output']  # shape: (batch_size, hidden_size)
```

**Get token vectors for sequence labeling**

```python
token_embeddings = bert_results['sequence_output']  # shape: (batch_size, seq_len, hidden_size)
```

**Mask-aware mean pooling across tokens**

```python
import tensorflow as tf
mask = tf.cast(text_preprocessed['input_mask'], tf.float32)  # shape (batch, seq_len)
emb = bert_results['sequence_output']  # (batch, seq_len, hidden_size)
masked_emb = emb * tf.expand_dims(mask, axis=-1)
sum_emb = tf.reduce_sum(masked_emb, axis=1)
lengths = tf.reduce_sum(mask, axis=1, keepdims=True)
mean_pooled = sum_emb / tf.maximum(lengths, 1.0)
# mean_pooled has shape (batch, hidden_size)
```

**Use encoder layer outputs**

```python
# encoder_outputs is an iterable of layer outputs
all_layers = bert_results['encoder_outputs']
# all_layers[i].shape == (batch, seq_len, hidden_size)
```


## 8. Practical tips and gotchas

* **Use the matching preprocessor for a model.** The tokenizer/vocabulary and preprocessing assumptions must match the encoder (WordPiece vocabulary, casing). The preprocessor you used (`bert_en_uncased_preprocess/3`) matches the uncased encoder.

* **Max sequence length and padding.** Default preprocessors pad or truncate to a fixed `max_seq_length` (128 here). If your task needs longer contexts, choose a model and preprocessor that support longer lengths or handle long text by chunking.

* **Fine-tuning vs. feature-extraction.** For best performance on a downstream task, fine-tune the encoder (unfreeze gradients). For quick experiments, using `pooled_output` as a frozen feature is simpler.

* **Token alignment.** If you need to map original words to tokens (for token-level labeling), remember WordPiece may split words into sub-tokens. Decide on a strategy: map labels to the first sub-token, average sub-token vectors, or use the original tokenizer to align.

* **Versioning.** TF Hub modules have versions. If reproducibility is important, pin the exact module version (you already did by using `/4` and `/3`).

---

## 9. Summary (short)

* Preprocessing converts raw text into `input_word_ids`, `input_mask`, and `input_type_ids`.
* The encoder returns token-level vectors (`sequence_output`), a pooled sentence vector (`pooled_output`), and intermediate layer outputs (`encoder_outputs`).
* Use `pooled_output` for sentence-level tasks and `sequence_output` for token-level tasks. `encoder_outputs` lets you inspect representations from every Transformer layer.
